# Jonathan Halverson
# Wednesday, September 21, 2016
# Accumulators and broadcast variables

In [57]:
lines = sc.textFile('text_file.md')
lines.take(5)

[u'# Apache Spark',
 u'',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 u'supports general computation graphs for data analysis. It also supports a']

In [58]:
pcount = sc.accumulator(0)
tcount = sc.accumulator(0)

In [59]:
def countPython(line):
    global pcount, tcount
    if ('python' in line.lower()): pcount += 1
    if ('the' in line.lower()): tcount += 1

In [60]:
lines.foreach(countPython)
print lines.first()
print pcount.value, tcount.value

# Apache Spark
3 21


Note that "grep -ci 'The' text_file.md" gives 21 and 3 in the case of 'Python'.

It is important to carry out an action such as first() on the RDD before evaluating the accumulators. This is because the transformation is not executed until the action is called.

In [63]:
keywords = ['code', 'hardware', 'causality', 'engine', 'computer']
ret = sc.broadcast(keywords)

In [67]:
out = lines.filter(lambda x: any([keyword in x for keyword in keywords]))
print out.count()

1


In [68]:
print out.collect()

[u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that']
